1. 침수심 csv 에서 구별, 날짜, 침수심(y) 열 데이터를 불러온다.(기준)

In [ ]:
import pandas as pd

In [ ]:
#침수심_부산_수정.csv 데이터를 불러와서 기준 데이터만 뽑아 낸다.
data = pd.read_csv("침수심_부산_수정.csv", encoding='CP949')

#기준데이터를 저장할 데이터프레임 만들기
stnd = pd.DataFrame()
stnd['idx'] = data['순번']
stnd['구 별-key1'] = data['행정구역 코드(ADM_CD)'].str.split(" ",expand=True)[1]
stnd['날짜(년-월-일)-key2'] = data['작성 년월(WRITNG_YM)']
stnd['침수심(y)'] = data['침수심(FLUD_DPWT)']
stnd['침수심(y)'].isnull().value_counts()

In [ ]:
stnd['침수심(y)']=stnd['침수심(y)'].fillna(0.001)
stnd

2. 강수량 csv 에서 구별, 강수량 열 데이터를 불러온다.

In [ ]:
#부산_일강수량_평균.csv 불러오기
rain = pd.read_csv("부산_일강수량_평균.csv", encoding='CP949')

#null 값 0으로 채우기
rain['강수량(mm)'] = rain['강수량(mm)'].fillna(0)

#필요 없는 열 삭제
rain = rain.drop('지점', axis=1)

#stnd, rain 합치기
#stnd:'날짜(년-월-일)-key2' 와 rain:'날짜' 같은 것끼리 연결
merge_rain = stnd.merge(rain, left_on='날짜(년-월-일)-key2', right_on='날짜')

#'날짜'열 삭제
merge_rain = merge_rain.drop('날짜', axis=1)
#열 이름 바꾸기
merge_rain = merge_rain.rename(columns={'강수량(mm)':'강수량(일 평균)'})
merge_rain

3. 하수도 보급률 csv에서 인구, 면적, 구별 열 데이터 불러온다.

4. 하수도 보급률 csv에서 지류, 구 열 뽑기

In [ ]:
#하수도 보급률_부산만.csv 불러오기
swer = pd.read_csv("하수도 보급률_부산만.csv", encoding='CP949')

#인구, 면적, 구별 열 데이터 불러온다.
swer = swer[['시도', '구군', '총인구(명)', '총면적']]


#새로운 인구밀도 열 생성 후 (인구/면적) 값 넣기
pd.options.display.float_format = '{:.5f}'.format #지수표현식으로 나오지 않도록 설정
swer['인구밀도'] = swer['총인구(명)'] / swer['총면적']

#인구, 면적, 시도 열 삭제
swer = swer.drop(['총인구(명)', '총면적', '시도'], axis=1)

#구별 , 인구밀도 데이터를 기준데이터에 맞춰 병합   
swer = swer[['구군', '인구밀도']]

#swer, merge_rain 합치기
#swer:'구 별-key1' 와 merge_rain:'시도구군' 같은 것끼리 연결
merge_swer = merge_rain.merge(swer, left_on='구 별-key1', right_on='구군', how = 'outer')

#'구군' 열 삭제
merge_swer = merge_swer.drop('구군', axis=1)

#열 이름 바꾸기
merge_swer = merge_swer.rename(columns={'인구밀도':'인구밀도(인구/면적)'})

#NaN값이 있으면 제거하기
merge_swer = merge_swer.dropna(subset=['구 별-key1'], how='any', axis=0)

merge_swer

5. 하수도_인구_및_보급률 csv에서 보급률과 구 컬럼 추출 후 기준데이터에 병합

In [ ]:
#하수도_인구_및_보급률(2009-2020).csv 불러오기
swer2 = pd.read_csv("하수도_인구_및_보급률(2009-2020).csv", encoding='CP949')

#날짜 비교를 위해 년도만 문자열 슬라이싱
merge_swer['년'] = merge_swer['날짜(년-월-일)-key2'].str.slice(start=0, stop=4)
merge_swer['년'] = merge_swer['년'].fillna(0)

#'구군별(1)'과 '하수도 보급률 (%)'만 추출
swer2 = swer2[['시점', '구군별(1)', '하수도 보급률 (%)']]

#swer2, merge_swer 합치기
#swer2:'구군별(1)' 와 merge_swer:'구 별-key1' && swer2: 같은 것끼리 연결
merge_swer['년'] = merge_swer['년'].astype(int)
merge_swer2 = merge_swer.merge(swer2, left_on=['구 별-key1', '년'], right_on=['구군별(1)', '시점'], how = 'outer')

#NaN값이 있으면 제거하기
merge_swer2 = merge_swer2.dropna(subset=['idx'], how='any', axis=0)


#2021년 하수도 보급률 데이터를 2020년 데이터로 채우기
swer2_2021 = swer2[swer2['시점'] == 2020]
swer2_2021['시점'] = swer2_2021['시점'].replace(2020, 2021)

#merge_swer, swer2_2021 합치기
#merge_swer:'구 별-key1' 와 swer2_2021:'구군별(1)' &&  merge_swer:'년'과 swer2_2021:'시점'같은 것끼리 연결
merge_swer2 = merge_swer2.merge(swer2_2021, left_on=['구 별-key1', '년'], right_on=['구군별(1)', '시점'], how = 'outer')

#NaN값이 있으면 제거하기
merge_swer2 = merge_swer2.dropna(subset=['idx'], how='any', axis=0)

#2021년 빈 데이터 값 채우기
merge_swer2['하수도 보급률 (%)_x'] = merge_swer2['하수도 보급률 (%)_x'].fillna(merge_swer2['하수도 보급률 (%)_y'])

#안 쓰는 열 삭제
merge_swer2 = merge_swer2.drop(['년', '시점_x', '시점_y', '구군별(1)_x','구군별(1)_y', '하수도 보급률 (%)_y'], axis=1)
merge_swer2 = merge_swer2.rename(columns={'하수도 보급률 (%)_x':'하수도 보급률 (%)'})
merge_swer2

6. 하수도 보급률 csv에서 구 , 설치율 컬럼 추출 후 기준데이터에 병합

In [ ]:
#하수도 보급률_부산만.csv 불러오기
swer3 = pd.read_csv("하수도 보급률_부산만.csv", encoding='CP949')
swer3 = swer3[['구군','총면적','하수도설치율','하수처리구역 내 면적']]

swer3['하수도설치율']=swer3['하수도설치율']/swer3['총면적']

merge_swer3 = merge_swer2.merge(swer3, left_on='구 별-key1', right_on='구군', how = 'outer')

#'구군' 열 삭제
merge_swer3 = merge_swer3.drop('구군', axis=1)
merge_swer3 = merge_swer3.drop('총면적', axis=1)

#열 이름 바꾸기
merge_swer3 = merge_swer3.rename(columns={'하수도설치율':'하수도 설치율(%)/총면적'})

#NaN값이 있으면 제거하기
merge_swer3 = merge_swer3.dropna(subset=['idx'], how='any', axis=0)

merge_swer3

7. 배수 펌프장 서비스 csv 에서 배수면적, 처리능력, 구 데이터 추출

In [ ]:
#한국국토정보공사_배수펌프장 서비스_수정.csv 불러오기
p_svc = pd.read_csv("부산광역시_배수 펌프장 현황_20220228.csv", encoding='CP949')

#구군, 배수량(제곱미터_분) 데이터 추출
p_svc = p_svc[['구군', '배수량(제곱미터_분)','유수지 용량(제곱미터)','유수지 면적(제곱미터)']]

p_svc['유수지 용량(제곱미터)']=p_svc['유수지 용량(제곱미터)']/p_svc['유수지 면적(제곱미터)']
#구별 평균 내기
mean = p_svc.groupby('구군')
mean = mean.mean().reset_index() #평균
mean['구군']  = mean['구군'].str.strip()  # 앞 뒤 공백을 제거

#데이터 합치기
#merge_swer3, mean 합치기
#mean:'구' 와 merge_swer3:'구 별-key1': 같은 것끼리 연결
merge_p_svc = merge_swer3.merge(mean, left_on='구 별-key1', right_on='구군', how ='outer')
merge_p_svc
merge_p_svc = merge_p_svc.drop(['구군','유수지 면적(제곱미터)'], axis=1)

#열 이름 바꾸기
merge_p_svc = merge_p_svc.rename(columns={'유수지 용량(제곱미터)':'유수지 면적 내 용량'})

#NaN값이 있으면 제거하기
merge_p_svc = merge_p_svc.dropna(subset=['idx'], how='any', axis=0)

#NaN은 0으로 채우기
merge_p_svc['배수량(제곱미터_분)'] = merge_p_svc['배수량(제곱미터_분)'].fillna(merge_p_svc['배수량(제곱미터_분)'].mean())
merge_p_svc['유수지 면적 내 용량'] = merge_p_svc['유수지 면적 내 용량'].fillna(merge_p_svc['유수지 면적 내 용량'].mean())
merge_p_svc

In [ ]:
#한국국토정보공사_배수펌프장 서비스_수정.csv 불러오기
typ = pd.read_csv("태풍목록(2009-2021)_수정.csv", encoding='CP949')


#날짜, 태풍 이름 추출
typ = typ[['태풍명', '발생']]

#'발생' 열 가공
typ['발생'] = typ['발생'].str.replace('.', '-')

#merge_swer3, p_svc 합치기
#typ:'발생' 과 merge_p_svc :'날짜(년-월-일)-key2': 같은 것끼리 연결
merge_typ = merge_p_svc .merge(typ, left_on='날짜(년-월-일)-key2', right_on='발생', how = 'outer')

#'발생' 열 삭제
merge_typ = merge_typ.drop('발생', axis=1)

#NaN값이 있으면 제거하기
merge_typ = merge_typ.dropna(subset=['idx'], how='any', axis=0)

#태풍 열 이름 바꾸기 : '태풍명' => '태풍 발생 여부'
merge_typ = merge_typ.rename(columns={'태풍명':'태풍 발생 여부'})

#NAN값을 제외한 태풍 확인
merge_typ['태풍 발생 여부'].dropna()

8. 태풍 csv에서 날짜, 태풍 이름 컬럼 추출 후 기준 데이터에 합치기

In [ ]:
# #merge_typ[merge_typ['태풍 발생 여부'].empty, : ]
# #태풍 이름이 있으면 1로 채우기
# if merge_typ['태풍 발생 여부'] != merge_typ['태풍 발생 여부'].isnull():
#   merge_typ['태풍 발생 여부'] = 0
#태풍 이름이 있으면 1로 채우기
merge_typ['태풍 발생 여부'] = merge_typ['태풍 발생 여부'].replace('하구핏', 1)

#태풍 발생하지 않은 행(NaN)은 0으로 채우기
merge_typ['태풍 발생 여부'] = merge_typ['태풍 발생 여부'].fillna(0)
merge_typ['태풍 발생 여부'] = pd.to_numeric(merge_typ['태풍 발생 여부'])

In [ ]:
# #merge_typ[merge_typ['태풍 발생 여부'].empty, : ]
# #태풍 이름이 있으면 1로 채우기
# if merge_typ['태풍 발생 여부'] != merge_typ['태풍 발생 여부'].isnull():
#   merge_typ['태풍 발생 여부'] = 0
#태풍 이름이 있으면 1로 채우기
merge_typ['태풍 발생 여부'] = merge_typ['태풍 발생 여부'].replace('하구핏', 1)

#태풍 발생하지 않은 행(NaN)은 0으로 채우기
merge_typ['태풍 발생 여부'] = merge_typ['태풍 발생 여부'].fillna(0)
merge_typ['태풍 발생 여부'] = pd.to_numeric(merge_typ['태풍 발생 여부'])

In [ ]:
import numpy as np

#침수심_부산_수정.csv 불러오기
cause = pd.read_csv("침수심_부산_수정.csv", encoding='CP949')

#순번, 침수 재해 명(F_DSSTR_NM), 침수 원인 상세(F_CUE_DTL) 추출
cause = cause[['순번', '침수 재해 명(F_DSSTR_NM)', '침수 원인 상세(F_CUE_DTL)']]

temp = cause[cause['침수 재해 명(F_DSSTR_NM)'].str.contains('태풍')|cause['침수 원인 상세(F_CUE_DTL)'].str.contains('태풍')]
temp2 = temp['순번']
temp2= temp2.to_frame()
temp2['태풍 여부'] = 1

#merge_swer3, p_svc 합치기
#typ:'발생' 과 merge_p_svc :'날짜(년-월-일)-key2': 같은 것끼리 연결
merge_typ2 = merge_typ.merge(temp2, left_on='idx', right_on='순번', how = 'outer')

#기존에 있던 태풍 발생 여부 데이터랑 합치기 : NaN 값이 있는 곳에 기존 데이터 채워넣기
merge_typ2['temp'] = np.where(pd.notnull(merge_typ2['태풍 여부']) ==True, merge_typ2['태풍 여부'], merge_typ2['태풍 발생 여부'])

#필요 없는 열 삭제
merge_typ2 = merge_typ2.drop(['순번', '태풍 발생 여부', '태풍 여부'], axis=1)
merge_typ2 = merge_typ2.rename(columns={'temp':'태풍 발생 여부'})

#Y:침수심, X1:강수량, X2:인구밀도, X3:하수도보급률, X4:하수도설치율/면적, X5:하수처리구역내면적, X6:배수량, X7:유수지용량, X8:태풍발생여부
merge_typ2=merge_typ2.drop(['idx','구 별-key1','날짜(년-월-일)-key2'],axis=1)
merge_typ2.columns = ['Y','X1','X2','X3','X4','X5','X6','X7','X8']

merge_typ2=merge_typ2.replace(0,0.001)


정규화

In [ ]:
import pandas as pd
import io
from sklearn.preprocessing import MinMaxScaler

df_n = pd.read_csv('전처리완료데이터.csv', encoding="cp949")

scaler = MinMaxScaler()
df_n.iloc[:,range(5,10)]=scaler.fit_transform(df_n.iloc[:,range(5,10)])

df_n.to_csv('정규화.csv', encoding='euc-kr',index=False)
df_n
